In [1]:
import math
import csv
from numpy import genfromtxt
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict
import pandas as pd

%matplotlib inline
np.random.seed(1)

In [2]:
# the original dataset as numpy matrix 
XY_orig=genfromtxt('14_day_churn_data_2.csv', delimiter=',') 
XY_orig=np.matrix(XY_orig)   

# taking out the X portion
X_orig=XY_orig[1:30001, 1:17].T   
X_orig_max = X_orig.max(1)          
X_orig=X_orig/X_orig_max
print ("X_orig shape: " + str(X_orig.shape))

# taking out the Y portion
Y_orig=XY_orig[1:30001, 17].T       

# training set
X_train=X_orig[:, 0:25000]          
Y_train=Y_orig[:, 0:25000]          

# test set
X_test=X_orig[:, 25000:30001]       
Y_test=Y_orig[:, 25000:30001]       

# convert all examples to numpy arrays
X_train=np.array(X_train)           
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

# convert to one-hot representations
Y_train=Y_train.astype(int)         
Y_test=Y_test.astype(int)
Y_train=convert_to_one_hot(Y_train, 2)
Y_test=convert_to_one_hot(Y_test, 2)

# printing out dimensions
print ("number of training examples = " + str(X_train.shape[1]))      
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

# printing out details
print("NaNs exist in training set: " + str(np.isnan(X_train).any()))
print("NaNs exist in test set: " + str(np.isnan(X_test).any()))

X_orig shape: (16, 30000)
number of training examples = 25000
number of test examples = 5000
X_train shape: (16, 25000)
Y_train shape: (2, 25000)
X_test shape: (16, 5000)
Y_test shape: (2, 5000)
NaNs exist in training set: False
NaNs exist in test set: False


In [3]:
# taking out the mobile numbers 
df_mobile_nos=pd.read_csv("14_day_churn_data_2.csv") 
mobile_nos=list(df_mobile_nos["mobile_no"])

# users with differently predicted retentions
pred_0=[]
pred_1=[]

In [4]:
def create_placeholders(n_x, n_y):

    X = tf.placeholder(tf.float32, [n_x, None])
    Y = tf.placeholder(tf.float32, [n_y, None])
    
    return X, Y

In [5]:
def initialize_parameters():
    
    # keeping results consistent
    tf.set_random_seed(1)                   
        
    # defining the neural network's architecture
    W1 = tf.get_variable("W1", [20, 16], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [20, 1], initializer = tf.zeros_initializer())
    
    W2 = tf.get_variable("W2", [25, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable("b2", [25, 1], initializer = tf.zeros_initializer())
    
    W3 = tf.get_variable("W3", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable("b3", [25, 1], initializer = tf.zeros_initializer())
    
    W4 = tf.get_variable("W4", [25, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b4 = tf.get_variable("b4", [25, 1], initializer = tf.zeros_initializer())
    
    W5 = tf.get_variable("W5", [20, 25], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b5 = tf.get_variable("b5", [20, 1], initializer = tf.zeros_initializer())
    
    W6 = tf.get_variable("W6", [2, 20], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b6 = tf.get_variable("b6", [2, 1], initializer = tf.zeros_initializer())

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  "W4": W4,
                  "b4": b4,
                  "W5": W5,
                  "b5": b5,
                  "W6": W6,
                  "b6": b6}
    
    return parameters

In [6]:
def forward_propagation(X, parameters):
    
    # retrieving the parameters from the dictionary 'parameters' 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    W4 = parameters['W4']
    b4 = parameters['b4']
    W5 = parameters['W5']
    b5 = parameters['b5']
    W6 = parameters['W6']
    b6 = parameters['b6']
                                         
    Z1 = tf.add(tf.matmul(W1, X), b1)                                              
    A1 = tf.nn.relu(Z1)   
    
    Z2 = tf.add(tf.matmul(W2, A1), b2)                                             
    A2 = tf.nn.relu(Z2)  
    
    Z3 = tf.add(tf.matmul(W3, A2), b3) 
    A3 = tf.nn.relu(Z3)
    
    Z4 = tf.add(tf.matmul(W4, A3), b4)
    A4 = tf.nn.relu(Z4)
    
    Z5 = tf.add(tf.matmul(W5, A4), b5)
    A5 = tf.nn.relu(Z5)
    
    Z6 = tf.add(tf.matmul(W6, A5), b6)
    
    return Z6

In [7]:
def compute_cost(Z6, Y):
    
    # to fit the tensorflow requirement for tf.nn.softmax_cross_entropy_with_logits(...,...)
    logits = tf.transpose(Z6)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    return cost

In [10]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.01,
          num_epochs = 500, minibatch_size = 64, print_cost = True):
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # n_x: input size, m : number of examples in the train set
    n_y = Y_train.shape[0]                            # n_y: output size
    costs = []                                        # to keep track of the cost
    
    # creating Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)

    # initializing parameters
    parameters = initialize_parameters()
    
    # forward propagation: building the forward propagation in the tensorflow graph
    Z6 = forward_propagation(X, parameters)
    
    # cost function: adding cost function to tensorflow graph
    cost = compute_cost(Z6, Y)
    
    # backpropagation: defining the tensorflow AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # initializing all the variables
    init = tf.global_variables_initializer()

    # starting the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # running the initialization
        sess.run(init)
        
        # training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # selecting a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                
                epoch_cost += minibatch_cost / num_minibatches

            # printing the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)

        # saving the parameters in a variable
        parameters = sess.run(parameters)
        
        print ("Parameters have been trained!")
        print()

        # accuracy
        correct_prediction = tf.equal(tf.argmax(Z6), tf.argmax(Y))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        print()
        
        # confusion matrix
        predicted0_actual0 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 0), tf.equal(tf.argmax(Y), 0)), "float"))
        predicted0_actual1 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 0), tf.equal(tf.argmax(Y), 1)), "float"))
        predicted1_actual0 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 1), tf.equal(tf.argmax(Y), 0)), "float"))
        predicted1_actual1 = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(tf.argmax(Z6), 1), tf.equal(tf.argmax(Y), 1)), "float"))

        print("True negatives:", predicted0_actual0.eval({X: X_test, Y: Y_test}))
        print("False negatives:", predicted0_actual1.eval({X: X_test, Y: Y_test}))
        print("False positives:", predicted1_actual0.eval({X: X_test, Y: Y_test}))
        print("True positives:", predicted1_actual1.eval({X: X_test, Y: Y_test}))
        print()

        # precision, recall, f1-score
        recall=tf.divide(predicted1_actual1, tf.add_n([predicted1_actual1, predicted0_actual1]))
        precision=tf.divide(predicted1_actual1, tf.add_n([predicted1_actual1, predicted1_actual0]))

        metric_train_prec=precision.eval({X: X_train, Y: Y_train})
        print("Train Precision:", metric_train_prec)
        metric_test_prec=precision.eval({X: X_test, Y: Y_test})
        print("Test Precision:", metric_test_prec)
        
        metric_train_rec=recall.eval({X: X_train, Y: Y_train})
        print("Train Recall:", metric_train_rec)
        metric_test_rec=recall.eval({X: X_test, Y: Y_test})
        print("Test Recall:", metric_test_rec)
        print()
        
        metric_train_f1=(2*metric_train_prec*metric_train_rec)/(metric_train_prec+metric_train_rec)
        print("Train f1:", metric_train_f1)
        metric_test_f1=(2*metric_test_prec*metric_test_rec)/(metric_test_prec+metric_test_rec)
        print("Test f1:", metric_test_f1)
        print()
        
        # predictions against test-set
        predicted0=tf.equal(tf.argmax(Z6), 0)
        predicted1=tf.equal(tf.argmax(Z6), 1)
        
        predicted_zeros=list(predicted0.eval({X: X_test}))
        len_predicted_zeros=len(predicted_zeros)
        for i in range(0, len_predicted_zeros):
            if(predicted_zeros[i]==True): pred_0.append(mobile_nos[i])
                
        predicted_ones=list(predicted1.eval({X: X_test}))
        len_predicted_ones=len(predicted_ones)
        for i in range(0, len_predicted_ones):
            if(predicted_ones[i]==True): pred_1.append(mobile_nos[i])
        
        return parameters

In [11]:
parameters = model(X_train, Y_train, X_test, Y_test)

Cost after epoch 0: 0.457689
Cost after epoch 100: 0.435188
Cost after epoch 200: 0.433454
Cost after epoch 300: 0.436979
Cost after epoch 400: 0.429971
Parameters have been trained!

Train Accuracy: 0.79644
Test Accuracy: 0.7908

True negatives: 1838.0
False negatives: 365.0
False positives: 681.0
True positives: 2116.0

Train Precision: 0.7665844
Test Precision: 0.75652486
Train Recall: 0.84937674
Test Recall: 0.8528819

Train f1: 0.8058597006831238
Test f1: 0.801818910306344



In [12]:
print(str(len(pred_0)) + " users predicted not to return in the next 14 days are:\n" + str(pred_0))
print()

print(str(len(pred_1)) + " users predicted to return in the next 14 days are:\n" + str(pred_1))
print()

2203 users predicted not to return in the next 14 days are:
[1715688392, 1835143173, 1778017444, 1738211889, 1717196083, 1990425436, 1710752631, 1904370991, 1794951718, 1846573373, 1722770077, 1766020898, 1756290289, 1729977491, 1722196999, 1833832028, 1718667834, 1945659578, 1927251890, 1305254628, 1839337928, 1918193733, 1611356593, 1671222244, 1719705374, 1728151318, 1749983319, 1724142620, 1982878740, 1790067003, 1990008880, 1976932603, 1818130707, 1749051933, 1737650730, 1864143949, 1690162452, 1747223718, 1813399182, 1613508988, 1681649072, 1310819029, 1922359906, 1319677487, 1713928606, 1789351356, 1400786010, 1685034532, 1404232620, 1621642490, 1303494155, 1720203903, 1733402030, 1790313737, 1751566727, 1892642770, 1713824277, 1717398875, 1844862151, 1723500365, 1937631357, 1764168923, 1761468117, 1892994123, 1575632214, 1829490783, 1686350692, 1717406458, 1719674977, 1316842256, 1736236217, 1768320246, 1308041656, 1916541865, 1723142941, 1845111947, 1917068571, 1929580476, 171